<div aling='center'>

# Entrega 1

</div>

### Enunciado
Ha sido contratado para el análisis de uso de un videojuego online donde millones de jugadores se conectan a diario para combatir con otros jugadores. En un combate, un jugador “retador” elije un oponente (jugador “retado”) y como resultado del combate el retador obtiene un puntaje y el tiempo en segundos que duró el combate. Por cada combate, los servidores del videojuego almacena la siguiente información:

- ID_Jugador_Retador
- ID_Jugador_Retado
- Puntaje obtenido (por el retador)
- Tiempo del combate en segundos

Todos los jugadores pueden participar cuántas veces deseen y por cada combate obtienen un puntaje y también se registra el tiempo que duró el combate. Un jugador puede combatir todas las veces que quiera.

Un fragmento del dataset de este jugador podría ser:

| ID_Jug_Retador | ID_Jugador_Retado | Puntos | Tiempo |
|---------------|-------------------|--------|--------|
| 231            | 492                | 1054   | 621    |
| 231            | 492                | 2068   | 504    |
| 231            | 98                 | 789    | 302    |
| 492            | 501                | 5462   | 955    |

Semanalmente los administradores del juego quieren obtener algunas estadísticas para regalarle premios a los mejores jugadores y armar el ranking semanal. Al finalizar la semana se desea saber:
### 1) El jugador más “retador” y el jugador más “retado”.

---

### 2) El jugador que más puntos obtuvo en promedio

$$
PP_i = \frac{puntaje\_total\_de\_todos\_los\_combates_i + 1}{cantidad\_de\_combates\_como\_retador_i + 1}
$$

*(el +1 en el numerador y en el denominador es para evitar divisiones por cero en el caso que un jugador no haya “retado” a nadie, en cuyo caso tendrá el puntaje mínimo de 1).*

---

### 3) Todos los jugadores que “retaron” a más de **H** oponentes **distintos**

*(H es un parámetro de la consulta).*

---

### 4) El top 10 de los jugadores con mejor puntaje heroico

El puntaje **PH** de un jugador *i* se calcula como:

$$
PH_i = \alpha \left( \sum_{j=1}^{n} PH_j \cdot \frac{PP_i}{PP_j} \right) + (1 - \alpha)
$$

donde:

- *n* es la cantidad de jugadores “retados” por el jugador *i*.  
- *PH_j* es el puntaje **P** del *j*-ésimo jugador retado por *i*.  
- *PP_i* es el puntaje promedio del jugador *i*.  
- *PP_j* es el puntaje promedio del *j*-ésimo jugador retado por *i*.  
- *α* es un parámetro del algoritmo.

### Nota sobre el cálculo del puntaje heroico (PH)

Como para calcular el puntaje **PH** de un jugador *i* previamente hay que calcular el puntaje de los jugadores a quiénes retó (y a su vez éstos pueden haber retado al jugador *i*), el cálculo se realiza de manera **iterativa**.

Se comienza asignando el mismo puntaje heroico con un valor arbitrario a todos los jugadores (**tiempo 0**).  
Con esos puntajes arbitrarios se calcula el puntaje usando la fórmula detallada para obtener los nuevos puntajes (**tiempo 1**),  
con éstos se calculan los nuevos puntajes (**tiempo 2**) y así se continúa hasta alcanzar una **convergencia en los valores**.

> 📌 Al final de este documento hay un ejemplo que explica este cálculo iterativo.


---

### Implementación

Implemente la cantidad de **jobs** necesarios para resolver las cuatro consultas.  
Utilice el archivo `jugadores.txt` para realizar las pruebas *(Se ofrecen los resultados de cada consulta)*.

---

### ⚠️ Aclaración

Suponga que la cantidad de jugadores y de combates es **“Big Data”**.

En cada **job** planteado en la solución, piense si una función **combiner** contribuye o no para la optimización del job.  
En caso de contribuir, implemente dicha función.

---


In [19]:
import sys
sys.path.append("..") 
from MRE import Job

inputDir = "../Datasets/TP1/input/"
outputDir = "../Datasets/TP1/out_punto1/"

def fmap(key, value, context):
    cols = value.strip().split()
    id_retador = key
    id_retado  = cols[0]
    context.write("RETADOR", id_retador)
    context.write("RETADO",  id_retado)

def fred(key, values, context):
    conteos = dict()
    for jugador in values:
        conteos[jugador] = conteos.get(jugador, 0) + 1
    jugador_max = max(conteos, key=conteos.get)
    context.write(key, (jugador_max, conteos[jugador_max]))

job = Job(inputDir, outputDir, fmap, fred)
success = job.waitForCompletion()

[MRE] INICIANDO ETAPA DE MAPEO...
[Map] RETADOR -> 15
[Map] RETADO -> 89
[Map] RETADOR -> 65
[Map] RETADO -> 51
[Map] RETADOR -> 82
[Map] RETADO -> 93
[Map] RETADOR -> 63
[Map] RETADO -> 33
[Map] RETADOR -> 103
[Map] RETADO -> 33
... [más resultados de map omitidos]
[MRE] INICIANDO ETAPA DE REDUCCIÓN...
[Reduce] Clave recibida: RETADO -> ['89', '51', '93', '33', '33', '17', '62', '70', '28', '66', '...']
[Reduce] Clave recibida: RETADOR -> ['15', '65', '82', '63', '103', '76', '3', '66', '3', '29', '...']
[MRE] FINALIZANDO Y ESCRIBIENDO RESULTADOS EN DISCO...

[MRE] RESULTADOS FINALES DEL JOB:
RETADO	('43', 15)
RETADOR	('1', 18)


In [20]:
#
import sys
sys.path.append("..") 
from MRE import Job

inputDir = "../Datasets/TP1/input/"
outputDir = "../Datasets/TP1/out_punto2/"

# ========== MAP ==========
def fmap2(key, value, context):
    cols = value.strip().split()
    id_retador = key
    puntaje = int(cols[1]) if len(cols) == 3 else int(cols[2])
    context.write(id_retador, (puntaje, 1))

# ========== REDUCE ==========
def fred2(key, values, context):
    total_puntos, total_combates = 0, 0
    for puntos, combates in values:
        total_puntos += puntos
        total_combates += combates
    # fórmula con +1
    promedio = (total_puntos + 1) / (total_combates + 1)
    context.write("PROMEDIO", (key, promedio))

# ========== JOB ==========
job = Job(inputDir, outputDir, fmap2, fred2)
success = job.waitForCompletion()

# ========== POST-PROCESO ==========
import os
mejor = ("", -1)
with open(os.path.join(outputDir, "output.txt")) as f:
    for line in f:
        _, jugador, promedio = line.strip().split("\t")
        promedio = float(promedio)
        if promedio > mejor[1]:
            mejor = (jugador, promedio)

print("\nConsulta 2:")
print(f"El jugador con más puntos en promedio: {mejor[0]} con {mejor[1]:.2f}")


[MRE] INICIANDO ETAPA DE MAPEO...
[Map] 15 -> (707, 1)
[Map] 65 -> (2411, 1)
[Map] 82 -> (3814, 1)
[Map] 63 -> (1033, 1)
[Map] 103 -> (4988, 1)
[Map] 76 -> (1692, 1)
[Map] 3 -> (1202, 1)
[Map] 66 -> (4625, 1)
[Map] 3 -> (2396, 1)
[Map] 29 -> (1867, 1)
... [más resultados de map omitidos]
[MRE] INICIANDO ETAPA DE REDUCCIÓN...
[Reduce] Clave recibida: 1 -> [(1896, 1), (2094, 1), (4432, 1), (4631, 1), (2548, 1), (1986, 1), (3657, 1), (2040, 1), (3308, 1), (827, 1), '...']
[Reduce] Clave recibida: 10 -> [(323, 1), (2905, 1), (749, 1)]
[Reduce] Clave recibida: 100 -> [(4529, 1)]
[Reduce] Clave recibida: 102 -> [(1220, 1), (2295, 1), (1641, 1), (2828, 1)]
[Reduce] Clave recibida: 103 -> [(4988, 1)]
[Reduce] Clave recibida: 104 -> [(4783, 1)]
[Reduce] Clave recibida: 11 -> [(1411, 1)]
[Reduce] Clave recibida: 113 -> [(3049, 1)]
[Reduce] Clave recibida: 116 -> [(3786, 1)]
[Reduce] Clave recibida: 12 -> [(3736, 1), (3886, 1), (1037, 1)]
... [más reduce claves omitidas]
[MRE] FINALIZANDO Y ESCRI

In [23]:
import sys
sys.path.append("..") 
from MRE import Job

inputDir = "../Datasets/TP1/input/"
outputDir = "../Datasets/TP1/out_punto3/"

# Parámetro H
H = 10

# ========== MAP ==========
def fmap3(key, value, context):
    cols = value.strip().split()
    if len(cols) < 1:
        return
    id_retador = key
    id_retado  = cols[0]
    context.write(id_retador, id_retado)

# ========== REDUCE ==========
def fred3(key, values, context):
    oponentes = set()
    for v in values:
        oponentes.add(v)
    cantidad = len(oponentes)
    context.write("DISTINTOS", (key, cantidad))

# ========== JOB ==========
job = Job(inputDir, outputDir, fmap3, fred3)
success = job.waitForCompletion()

# ========== POST-PROCESO ==========
import os
jugadores = []
with open(os.path.join(outputDir, "output.txt")) as f:
    for line in f:
        _, jugador, cantidad = line.strip().split("\t")
        cantidad = int(cantidad)
        if cantidad > H:
            jugadores.append(jugador)

print("\nConsulta 3:")
print(f"Con H = {H}")
print("Jugadores que retaron a más de H oponentes distintos:", jugadores)


[MRE] INICIANDO ETAPA DE MAPEO...
[Map] 15 -> 89
[Map] 65 -> 51
[Map] 82 -> 93
[Map] 63 -> 33
[Map] 103 -> 33
[Map] 76 -> 17
[Map] 3 -> 62
[Map] 66 -> 70
[Map] 3 -> 28
[Map] 29 -> 66
... [más resultados de map omitidos]
[MRE] INICIANDO ETAPA DE REDUCCIÓN...
[Reduce] Clave recibida: 1 -> ['73', '48', '49', '2', '37', '23', '48', '64', '47', '15', '...']
[Reduce] Clave recibida: 10 -> ['64', '8', '60']
[Reduce] Clave recibida: 100 -> ['67']
[Reduce] Clave recibida: 102 -> ['52', '53', '23', '81']
[Reduce] Clave recibida: 103 -> ['33']
[Reduce] Clave recibida: 104 -> ['61']
[Reduce] Clave recibida: 11 -> ['49']
[Reduce] Clave recibida: 113 -> ['27']
[Reduce] Clave recibida: 116 -> ['42']
[Reduce] Clave recibida: 12 -> ['48', '50', '25']
... [más reduce claves omitidas]
[MRE] FINALIZANDO Y ESCRIBIENDO RESULTADOS EN DISCO...

[MRE] RESULTADOS FINALES DEL JOB:
DISTINTOS	('1', 15)
DISTINTOS	('10', 3)
DISTINTOS	('100', 1)
DISTINTOS	('102', 4)
DISTINTOS	('103', 1)
DISTINTOS	('104', 1)
DISTINTOS

In [24]:
import sys
sys.path.append("..")
from MRE import Job

inputDir = "../Datasets/TP1/input/"
outputDir = "../Datasets/TP1/out_ppi/"

# ========== MAP ==========
def fmap_ppi(key, value, context):
    cols = value.strip().split()
    if len(cols) < 2:
        return
    id_retador = key
    puntos = int(cols[1]) if len(cols) == 3 else int(cols[2])
    context.write(id_retador, (puntos, 1))

# ========== REDUCE ==========
def fred_ppi(key, values, context):
    total_puntos, total_combates = 0, 0
    for p, c in values:
        total_puntos += p
        total_combates += c
    ppi = (total_puntos + 1) / (total_combates + 1)
    context.write("PPi", (key, ppi))

# ========== JOB ==========
job_ppi = Job(inputDir, outputDir, fmap_ppi, fred_ppi)
success = job_ppi.waitForCompletion()
print("Job 1 (PPi) terminado:", success)


[MRE] INICIANDO ETAPA DE MAPEO...
[Map] 15 -> (707, 1)
[Map] 65 -> (2411, 1)
[Map] 82 -> (3814, 1)
[Map] 63 -> (1033, 1)
[Map] 103 -> (4988, 1)
[Map] 76 -> (1692, 1)
[Map] 3 -> (1202, 1)
[Map] 66 -> (4625, 1)
[Map] 3 -> (2396, 1)
[Map] 29 -> (1867, 1)
... [más resultados de map omitidos]
[MRE] INICIANDO ETAPA DE REDUCCIÓN...
[Reduce] Clave recibida: 1 -> [(1896, 1), (2094, 1), (4432, 1), (4631, 1), (2548, 1), (1986, 1), (3657, 1), (2040, 1), (3308, 1), (827, 1), '...']
[Reduce] Clave recibida: 10 -> [(323, 1), (2905, 1), (749, 1)]
[Reduce] Clave recibida: 100 -> [(4529, 1)]
[Reduce] Clave recibida: 102 -> [(1220, 1), (2295, 1), (1641, 1), (2828, 1)]
[Reduce] Clave recibida: 103 -> [(4988, 1)]
[Reduce] Clave recibida: 104 -> [(4783, 1)]
[Reduce] Clave recibida: 11 -> [(1411, 1)]
[Reduce] Clave recibida: 113 -> [(3049, 1)]
[Reduce] Clave recibida: 116 -> [(3786, 1)]
[Reduce] Clave recibida: 12 -> [(3736, 1), (3886, 1), (1037, 1)]
... [más reduce claves omitidas]
[MRE] FINALIZANDO Y ESCRI

In [25]:
inputDir = "../Datasets/TP1/input/"
outputDir = "../Datasets/TP1/out_grafo/"

# ========== MAP ==========
def fmap_grafo(key, value, context):
    cols = value.strip().split()
    if len(cols) < 1:
        return
    id_retador = key
    id_retado = cols[0]
    context.write(id_retador, id_retado)

# ========== REDUCE ==========
def fred_grafo(key, values, context):
    # eliminamos duplicados con set
    oponentes = set()
    for v in values:
        oponentes.add(v)
    context.write("GRAFO", (key, list(oponentes)))

# ========== JOB ==========
job_grafo = Job(inputDir, outputDir, fmap_grafo, fred_grafo)
success = job_grafo.waitForCompletion()
print("Job 2 (Grafo) terminado:", success)


[MRE] INICIANDO ETAPA DE MAPEO...
[Map] 15 -> 89
[Map] 65 -> 51
[Map] 82 -> 93
[Map] 63 -> 33
[Map] 103 -> 33
[Map] 76 -> 17
[Map] 3 -> 62
[Map] 66 -> 70
[Map] 3 -> 28
[Map] 29 -> 66
... [más resultados de map omitidos]
[MRE] INICIANDO ETAPA DE REDUCCIÓN...
[Reduce] Clave recibida: 1 -> ['73', '48', '49', '2', '37', '23', '48', '64', '47', '15', '...']
[Reduce] Clave recibida: 10 -> ['64', '8', '60']
[Reduce] Clave recibida: 100 -> ['67']
[Reduce] Clave recibida: 102 -> ['52', '53', '23', '81']
[Reduce] Clave recibida: 103 -> ['33']
[Reduce] Clave recibida: 104 -> ['61']
[Reduce] Clave recibida: 11 -> ['49']
[Reduce] Clave recibida: 113 -> ['27']
[Reduce] Clave recibida: 116 -> ['42']
[Reduce] Clave recibida: 12 -> ['48', '50', '25']
... [más reduce claves omitidas]
[MRE] FINALIZANDO Y ESCRIBIENDO RESULTADOS EN DISCO...

[MRE] RESULTADOS FINALES DEL JOB:
GRAFO	('1', ['41', '37', '23', '43', '57', '48', '10', '2', '47', '49', '96', '42', '73', '15', '64'])
GRAFO	('10', ['60', '64', '8']

In [30]:
import sys, os
sys.path.append("..")
from MRE import Job

alpha = 0.85

# Directorios
ppi_file   = "../Datasets/TP1/out_ppi/output.txt"
grafo_file = "../Datasets/TP1/out_grafo/output.txt"
inputDir   = "../Datasets/TP1/ph_iter_in/"   # archivo con PH actual
outputDir  = "../Datasets/TP1/ph_iter_out/"

# ========== Cargar PPi ==========
def load_ppi(path):
    ppi = {}
    with open(path) as f:
        for line in f:
            parts = line.strip().split("\t", 1)
            if len(parts) < 2:
                continue
            _, raw = parts
            raw = raw.strip()
            try:
                # Caso: ('1', 1818.26)
                jugador, val = eval(raw)
                ppi[str(jugador)] = float(val)
            except:
                # Caso: 1    1818.26
                subparts = raw.split()
                if len(subparts) == 2:
                    jugador, val = subparts
                    ppi[str(jugador)] = float(val)
    return ppi

# ========== Cargar Grafo ==========
def load_grafo(path):
    grafo = {}
    with open(path) as f:
        for line in f:
            parts = line.strip().split("\t", 1)
            if len(parts) < 2:
                continue
            _, raw = parts
            raw = raw.strip()
            try:
                # Caso: ('1', ['41','37','23',...])
                jugador, lista = eval(raw)
                grafo[str(jugador)] = [str(x) for x in lista]
            except:
                # Caso alternativo: "1 41 37 23 ..."
                subparts = raw.split()
                if len(subparts) > 1:
                    jugador, *lista = subparts
                    grafo[str(jugador)] = lista
    return grafo

# ========== Cargar estructuras ==========
PPIs = load_ppi(ppi_file)
GRAFO = load_grafo(grafo_file)

print("Ejemplo PPi:", list(PPIs.items())[:5])
print("Ejemplo Grafo:", list(GRAFO.items())[:3])

# ========== MAP ==========
def fmap_ph(key, value, context):
    cols = value.strip().split()
    if len(cols) != 2:
        return
    jugador, ph_val = cols[0], float(cols[1])
    PPI_i = PPIs.get(jugador, 1.0)

    if jugador not in GRAFO:
        return

    for retado in GRAFO[jugador]:
        PPI_j = PPIs.get(retado, 1.0)
        contrib = ph_val * (PPI_i / PPI_j)
        context.write(retado, contrib)

# ========== REDUCE ==========
def fred_ph(key, values, context):
    suma = sum(values)
    nuevo_ph = alpha * suma + (1 - alpha)
    context.write("PH", (key, nuevo_ph))

# ========== JOB ==========
job_ph = Job(inputDir, outputDir, fmap_ph, fred_ph)
success = job_ph.waitForCompletion()
print("Iteración PH terminada:", success)



Ejemplo PPi: [('1', 1818.2631578947369), ('10', 994.5), ('100', 2265.0), ('102', 1597.0), ('103', 2494.5)]
Ejemplo Grafo: [('1', ['41', '37', '23', '43', '57', '48', '10', '2', '47', '49', '96', '42', '73', '15', '64']), ('10', ['60', '64', '8']), ('100', ['67'])]
[MRE] INICIANDO ETAPA DE MAPEO...
[Map] 41 -> 0.8333489122404981
[Map] 37 -> 1.0500749756835621
[Map] 23 -> 0.7154135009640916
[Map] 43 -> 0.5446930331348978
[Map] 57 -> 0.717232136266332
[Map] 48 -> 0.5455209105153839
[Map] 10 -> 1.8283189119102432
[Map] 2 -> 1818.2631578947369
[Map] 47 -> 0.8486972614546208
[Map] 49 -> 1.7231455249191971
... [más resultados de map omitidos]
[MRE] INICIANDO ETAPA DE REDUCCIÓN...
[Reduce] Clave recibida: 10 -> [1.8283189119102432]
[Reduce] Clave recibida: 15 -> [2.746621084433137]
[Reduce] Clave recibida: 2 -> [1818.2631578947369]
[Reduce] Clave recibida: 21 -> [1.426412429378531]
[Reduce] Clave recibida: 23 -> [0.7154135009640916]
[Reduce] Clave recibida: 28 -> [1.1317519611505416]
[Reduce] 